In [1]:
import pandas as pd
import csv
import sys
import requests
from lxml import html as htmlRenderer
csv.field_size_limit(sys.maxsize)

131072

In [12]:
csv_file_in = "/home/cflores/cflores_workspace/comments-retriever/results/elmundo/elmundo-01012019-15092019-contents-fixed.csv"
csv_file_out = "/home/cflores/cflores_workspace/comments-retriever/results/elmundo/elmundo-01012019-15092019-contents.csv"

In [5]:
def is_valid_csv(csv_file):
    keys = []
    no_content = []
    no_title = []
    with open(csv_file, 'r') as f:
        reader = csv.reader(f)
        for r in reader:
            if not r[1]:
                no_content.append(r[0])
                subkey = r[0].split("https://www.elmundo.es/")[1]
                key = subkey.split("/")[0]
                keys.append(key)
            if not r[2] or r[2] == r[0]:
                no_title.append(r[0])
        print(" \t rows without content: {}".format(len(no_content)))
        print(" \t urls key without content: {}".format(list(dict.fromkeys(keys))))
        print(" \t rows without title: {}".format(len(no_title)))
    
    return len(no_content) == len(no_title) == 0, no_content, no_title

# https://www.elmundo.es/deportes/tenis/2019/01/05/5c30db5e21efa057748b4684.html

In [7]:
def get_title(url):
    queries_xpath = ["//div[@class='ue-l-article__header']//h1[@class='js-headline']", 
                 "//div[@class='ue-l-article__header']//h1[@class='ue-c-article__headline js-headline']",
                "//div[@class='titles']//h1[@class='js-headline']",
                "//div[@class='ue-l-article__header']//h1",
                "//h1[@class='ue-c-article__headline js-headline ']",
                "//h1[@itemprop='headline']"]

    response = requests.get(url)
    renderedPage = htmlRenderer.fromstring(response.text)
    title = url
    for q in queries_xpath:
        el = renderedPage.xpath(q)
        if len(el) > 0:
            title = el[0].text
            break
    if title == url:
        print (" \t -> title not found for url {}".format(url))
    return title

In [8]:
def extract_content(url=None):
    queries_xpath = ["//div[@data-section='articleBody']/p", 
                     "//div[@class='row content cols-70-30']/p", 
                     "//dl[@class='ue-c-article__interview']",
                     "//ul[@class='ue-c-article__list ue-c-article__list--unordered']", 
                     "//dl[@class='interview']"]
    
    response = requests.get(url)
    renderedPage = htmlRenderer.fromstring(response.text)
    
    contentStr = ""
    for q in queries_xpath:
        commentsElList = renderedPage.xpath(q)
        if len(commentsElList) > 0:
            contentArr = []
            for p in commentsElList:
                contentArr.append(p.text_content())
            contentStr = "".join([parrafo for parrafo in contentArr])
            break
    if not contentStr:
        print("\t -> url has not content found {}".format(url))
    return contentStr

            
# content = {
#     "url": url,
#     "content": contentStr
# }
# return [content]

In [13]:
keys = []
no_content = []
no_title = []
forbidden_tag = "Conforme a los criterios de"
with open(csv_file_in, 'r') as f:
    with open(csv_file_out, 'w') as fout:
        writer = csv.writer(fout)
        reader = csv.reader(f)
        for r in reader:
            if not r[1]:
                # no content
                print(" => row with url {} has any content".format(r[0]))
                content = extract_content(r[0])
                if content:
                    print(" \t ===>>> content found ")
                r[1] = content
            else:
                r[1] = r[1][:r[1].find(forbidden_tag)]
            if not r[2] or r[2] == r[0]:
                # no title
                print(" => row with url {} has any title".format(r[0]))
                r[2] = get_title(r[0])
            
            writer.writerow(r)    

 => row with url https://www.elmundo.es/opinion/2019/01/01/5c2a4cb7fc6c83dc7f8b4625.html has any content
	 -> url has not content found https://www.elmundo.es/opinion/2019/01/01/5c2a4cb7fc6c83dc7f8b4625.html
 => row with url https://www.elmundo.es/television/2019/01/02/5c2c19ec21efa0d7438b4651.html has any content
	 -> url has not content found https://www.elmundo.es/television/2019/01/02/5c2c19ec21efa0d7438b4651.html
 => row with url https://www.elmundo.es/opinion/2019/01/03/5c2cfe6dfdddffbc1f8b45c1.html has any content
	 -> url has not content found https://www.elmundo.es/opinion/2019/01/03/5c2cfe6dfdddffbc1f8b45c1.html
 => row with url https://www.elmundo.es/opinion/2019/01/04/5c2e44ba21efa01e708b45c7.html has any content
	 -> url has not content found https://www.elmundo.es/opinion/2019/01/04/5c2e44ba21efa01e708b45c7.html
 => row with url https://www.elmundo.es/opinion/2019/01/05/5c2f5611fdddffbb1f8b4649.html has any content
	 -> url has not content found https://www.elmundo.es/opi

In [14]:
valid, without_content, without_title = is_valid_csv(csv_file_in)

 	 rows without content: 285
 	 urls key without content: ['opinion', 'television', 'espana', 'internacional', 'comunidad-valenciana', 'deportes', 'madrid', 'cultura', 'andalucia', 'pais-vasco', 'cataluna', 'baleares', 'papel']
 	 rows without title: 0


In [15]:
valid, without_content, without_title = is_valid_csv(csv_file_out)

 	 rows without content: 234
 	 urls key without content: ['opinion', 'television', 'espana', 'internacional', 'deportes', 'madrid']
 	 rows without title: 0


In [16]:
without_content

['https://www.elmundo.es/opinion/2019/01/01/5c2a4cb7fc6c83dc7f8b4625.html',
 'https://www.elmundo.es/television/2019/01/02/5c2c19ec21efa0d7438b4651.html',
 'https://www.elmundo.es/opinion/2019/01/03/5c2cfe6dfdddffbc1f8b45c1.html',
 'https://www.elmundo.es/opinion/2019/01/04/5c2e44ba21efa01e708b45c7.html',
 'https://www.elmundo.es/opinion/2019/01/05/5c2f5611fdddffbb1f8b4649.html',
 'https://www.elmundo.es/opinion/2019/01/07/5c3251a1fc6c8356098b4696.html',
 'https://www.elmundo.es/opinion/2019/01/07/5c32234ffc6c837c5d8b4684.html',
 'https://www.elmundo.es/opinion/2019/01/08/5c33978cfc6c83723a8b45d7.html',
 'https://www.elmundo.es/opinion/2019/01/09/5c34f18b21efa039548b4611.html',
 'https://www.elmundo.es/opinion/2019/01/09/5c34e8fa21efa0cd188b45f4.html',
 'https://www.elmundo.es/opinion/2019/01/10/5c3653a421efa0a8178b46bf.html',
 'https://www.elmundo.es/espana/2019/01/11/5c38895e21efa09d6c8b4664.html',
 'https://www.elmundo.es/espana/2019/01/12/5c39c63efdddff5c178b461d.html',
 'https://w